# SPLICE: Fully Tractable Hierarchical Extension of ICA with Pooling

In [1]:
import numpy as np

In [2]:
from scipy import signal, linalg

In [3]:
from matplotlib import pyplot as plt

In [4]:
%matplotlib widget

In [5]:
import seaborn as sns

In [6]:
sns.set()

In [7]:
from cycler import cycler
import matplotlib as mpl
mpl.rcParams['axes.prop_cycle'] = cycler(color=['red', 'steelblue', 'orange'])

In [8]:
sns.set_style('white')

## ICA

Let's take a look at ICA

Inspired from the sklearn documentation

In [9]:
from sklearn.decomposition import fastica

#### Generating toy data

In [10]:
np.random.seed(0)
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)                   # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))          # Signal 2 : square signal
s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal

S = np.c_[s1, s2, s3]
S += 0.2 * np.random.normal(size=S.shape) # Add noise
S /= S.std(axis=0)                        # Standardize data

# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
X = np.dot(S, A.T)                                         # Generate observations

In [11]:
plt.figure(figsize = (18, 3))
plt.plot(time, S)
plt.xlabel('time')
plt.legend(('source 1', 'source 2', 'source 3'))
plt.title('True sources')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.figure(figsize = (18, 3))
plt.plot(time, X)
plt.xlabel('time')
plt.legend(('channel 1', 'channel 2', 'channel 3'))
plt.title('Observations')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Computing ICA

In [13]:
K, W_, S_ = fastica(X, n_components=3)

- $K$: pre-whitening matrix
- $\hat{W}$: estimated un-mixing matrix
- $\hat{S}$: estimated source matrix

We can then calculate the estimated mixing matrix $\hat{A}$

In [14]:
A_ = linalg.pinv(np.dot(W_, K))

In [15]:
plt.figure(figsize = (18, 3))
plt.plot(time, S_)
plt.xlabel('time')
plt.legend(('estimated source 1', 'estimated source 2', 'estimated source 3'))
plt.title('Recovered signals')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### ICA from scratch

To make sure we understand the way ICA works, let's implement it from scratch

#### Visualization

Let's first take a look at our data

In [16]:
from mpl_toolkits.mplot3d import Axes3D

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

l = ax.scatter(X[:,0], X[:,1], X[:,2], c='blue')

ax.set_xlabel('observation 1')
ax.set_ylabel('observation 2')
ax.set_zlabel('observation 3')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is not very useful, let's get a better view

In [18]:
ax.view_init(elev=-35.83, azim=8.917)
plt.show()

We can see that there are two well-separated clusters.

Intuitively, this is caused by the binary signal in source 2.
We can check this intuition by coloring the graph

In [19]:
color = ['red' if s > 0 else 'blue' for s in S[:,1]]
l.set_color(color)
l._facecolor3d = l.get_facecolor()
l._edgecolor3d = l.get_edgecolor()

This means that variance within each of these clusters is caused by the mixing of sources 1 and 3

In [20]:
ax.view_init(elev=-26.09, azim=-26.63)
plt.show()

#### Whitening

Centering

In [23]:
def whitening(X):
    
    X -= X.mean(axis=0)            # Centering
    Cov = np.dot(X.T,X) / len(X)   # Calculating the covariance matrix
    U, s, Vh = linalg.svd(Cov)     # SVD of the covariance matrix
    S = np.sqrt(np.diag(1 / s))    
    W = np.dot(S, Vh)              # Whitening transformation
    Z = np.dot(X, W.T)    # Whitened observations
    
    return Z

In [24]:
Z = whitening(X)

In [40]:
plt.figure(figsize = (18, 3))
plt.plot(time, Z)
plt.xlabel('time')
plt.legend(('whitened observation 1', 'whitened observation 2', 'whitened observation 3'))
plt.title('Whitened observations')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

l = ax.scatter(Z[:,0], Z[:,1], Z[:,2], c='blue')

ax.set_xlabel('whitened observation 1')
ax.set_ylabel('whitened observation 2')
ax.set_zlabel('whitened observation 3')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
ax.view_init(elev=96.20, azim=45.12)
plt.show()

In [50]:
color = ['red' if s > 0 else 'blue' for s in S[:,1]]
l.set_color(color)
l._facecolor3d = l.get_facecolor()
l._edgecolor3d = l.get_edgecolor()

In [56]:
ax.view_init(elev=4.754, azim=50.35)
plt.show()

In [54]:
color = ['red' if s > 0 else 'blue' for s in S[:,0]]
l.set_color(color)
l._facecolor3d = l.get_facecolor()
l._edgecolor3d = l.get_edgecolor()

In [55]:
color = ['red' if s > 0 else 'blue' for s in S[:,2]]
l.set_color(color)
l._facecolor3d = l.get_facecolor()
l._edgecolor3d = l.get_edgecolor()

We can attest that the observations have been properly whitened.

Since they are now decorrelated, we can now apply the ICA.